In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import pandas as pd
import requests

In [75]:
url = 'https://nyheder.tv2.dk/seneste'
driver = webdriver.Chrome()
driver.get(url)

headers = {
    'Name': 'Mathias Udsen', 
    'E-mail':'lcv842@alumni.ku.dk',
    'Message': 'Scraping as a part of our exam project at UCPH, Intoduction to social data science' 
}
requests.get(url, headers)
#time.sleep(5)

<Response [200]>

In [76]:
cookie = driver.find_element(By.CSS_SELECTOR, '#onetrust-reject-all-handler')
cookie.click()
#time.sleep(5)

In [33]:
Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
Vis_flere.click()
#time.sleep(5)

In [5]:
"""
i = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
        Vis_flere.click()
        time.sleep(1)
        i = i + 1
    except:
        time.sleep(1)
"""

"\ni = 0\nwhile i <= 10:\n    try:\n        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')\n        Vis_flere.click()\n        time.sleep(1)\n        i = i + 1\n    except:\n        time.sleep(1)\n"

In [51]:
"""
i = 0
count_of_one_column_lists = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
        i += 1
    except:
        time.sleep(1)
"""

for i in range(5):
    print(len(soup.find_all('ul', {'data-columns': '1'})[i].find_all(class_='tc_teaser')))

5
5
19


IndexError: list index out of range

In [77]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

Headline = []
Category = []
Authors = []
Time = []
Live = []
All_text = []
nummer=0

Live_find1 = False
Live_find2 = False

max_retries = 5
retries = 0

Mathias = 'nej'

#max_iter = 1001
max_iter = 5
#i = 1000
i = 0
count_of_one_column_lists = 0
if Mathias == 'Ja':
    
    pbar1 = tqdm(total=max_iter, desc="Loading content")

    #while i < 2001:
    while i < 5:
        try:
            Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
            Vis_flere.click()

            new_count_of_one_column_lists = 0
            while new_count_of_one_column_lists <= count_of_one_column_lists:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
                time.sleep(0.25)

            count_of_one_column_lists = new_count_of_one_column_lists
            i += 1
            pbar1.update(1)
        except:
            time.sleep(1)
    pbar1.close()

pbar2 = tqdm(total=max_iter, desc="Scraping")
i = 0
#for i in range(1000, 2001, 1):
for i in range(10):
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
    except:
        time.sleep(1)
        
    retries = 0
    while retries < max_retries:
        try:
            one_column_lists = soup.find_all('ul', {'data-columns': '1'})[i]
            break
        except:
            #print(i)
            retries = retries + 1
            time.sleep(1)
    
    if retries == max_retries:
        print(f"Max tries at: Number {nummer} and i: {i}")
        continue
        
    pbar3 = tqdm(total=len(soup.find_all('ul', {'data-columns': '1'})[i].find_all(class_='tc_teaser')), desc="Articles", leave=False)

    for one_column_list in one_column_lists:
        artikler = one_column_list.find_all(class_='tc_teaser')
            
        for artikel in artikler:
            nummer = nummer + 1
            
            if artikel.find('span', class_='tc_label tc_label--color-live'):
                Live_find1 = True
            else:
                Live_find1 = False
            
            Category.append(artikel.find('span', class_='tc_label tc_label--color-nyheder').text)
            
            href = artikel.find('a', class_='tc_teaser__link').get('href') #if link else None

            text = []
            if href:
                driver.execute_script(f'window.open("{href}", "_blank");')
                driver.switch_to.window(driver.window_handles[-1])

                new_html = driver.page_source
                new_soup = BeautifulSoup(new_html, 'lxml')

                #Category.append(new_soup.find('div', class_='tc_page__label').find('a', class_='tc_label tc_label--color-nyheder').text)
                #Headline.append(new_soup.find('header', class_="tc_page__header").find('h1').text)
                
                liveblog_body = new_soup.find('div', class_="tc_liveblog__body", attrs={'data-adobe-context': "article-body"})
                if liveblog_body:
                    em_tag = liveblog_body.find('em')
                    if em_tag and em_tag.text == 'Denne liveblog opdateres ikke længere.':
                        Live_find2 = True
                    else:
                        Live_find2 = False
                else:
                    Live_find2 = False
                
                if Live_find1 == True or Live_find2 == True:
                    Headline.append(new_soup.find('header', class_="tc_page__header").find('h1').text)
                
                Authors.append(new_soup.find('div', class_="tc_byline").find('strong').text)
                Time.append(new_soup.find('time', class_='tc_timestamp').get('datetime'))
                
                if liveblog_body:
                    article_body_div = liveblog_body
                else:
                    article_body_div = new_soup.find('div', class_='tc_page__body', attrs={'data-adobe-context': 'article-body'})
                
                if article_body_div:
                    rich_content_paragraphs = article_body_div.find_all('p')

                    for paragraph in rich_content_paragraphs:
                        text.append(paragraph.text)

                All_text.append(text)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])
            
            if Live_find1 == True or Live_find2 == True:
                Live.append(1)
            else:
                Live.append(0)
            
            if Live_find1 == False and Live_find2 == False:
                Headline.append(artikel.find('a', class_='tc_teaser__link').get('aria-label'))
            pbar3.update(1)
    pbar3.close()
    pbar2.update(1)
pbar2.close()
time.sleep(5)

Scraping:   0%|          | 0/5 [00:00<?, ?it/s]

Articles:   0%|          | 0/5 [00:00<?, ?it/s]

Articles:   0%|          | 0/5 [00:00<?, ?it/s]

Articles:   0%|          | 0/19 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

...

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
print(len(Category), Category[0])
print(len(Authors), Authors[0])
print(len(Headline), Headline[0])
print(len(Time), Time[0])
print(len(Live), Live[0])
print(len(All_text), All_text[0])
#time.sleep(5)

129 Lokalt
129  Pernille Leftes
129 Jysk skole opretter 0. klasse med bare tre elever
129 2023-08-14T11:04:48.740Z
129 0
129 ['Mange spændte børn har mandag første skoledag i 0. klasse. Også i Ørslevkloster, hvor 0. klasse i år består af kun tre nye elever.', 'Petrina, Anna og Olivia. ', 'Navneopråbet på de nye elever i 0. klasse på skolen i Ørslevkloster er hurtig overstået, for årgangen består af kun tre piger. Det er dog ikke noget, der bekymrer Annas forældre.', '- Vi er slet ikke utrygge, fordi det personale, der er her, er virkelig gode til børnene. Og fagligheden fejler ikke noget, og de er gode til at se hvert enkelt barn, siger Mette Vestergaard Kristensen. ', 'De tre nye skoleelever skal dog ikke nøjes med hinandens selskab. Deres årgang bliver fra start slået sammen med både 1. og 2. klasse, så de bliver 23 elever på holdet:', '- Så lærer de jo af de ældre børn. Jeg tror faktisk, at det er en god idé i forhold til skolestart, at de er sammen med nogle, der har gået lidt i sk

In [78]:
data = {
    'Category': Category,
    'Authors': Authors,
    'Headline': Headline,
    'Time': Time,
    'Live': Live,
    'All_text': All_text
}
df = pd.DataFrame(data)
#time.sleep(5)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df
#sorted_df = df.sort_values(by='Time')
#sorted_df

,Category,Authors,Headline,Time,Live,All_text
378,Krigen i Ukraine,Mikkel Secher,"Testfasen er slut, siger Ukraine om droneangre...",2023-08-04T07:15:11.454Z,0,[Ukraine investerer massivt i at udvikle langt...
377,Udland,Ritzau,Loch Ness-uhyrets hjemsted er ramt af klimafor...,2023-08-04T07:16:56.291Z,0,[En tør start på året giver udfordringer ved L...
376,Lokalt,Niels Ettrup Brande,Familier i slagsmål på færge – flere sendt på ...,2023-08-04T07:30:13.510Z,0,"[To familier, der tilsyneladende har en uløst ..."
375,Business,Steffen Neupert,"Mærsks topchef kalder koranafbrændinger ""virke...",2023-08-04T07:38:53.275Z,0,[Vincent Clerc bakker op om et forbud mod kora...
374,Samfund,David Rue Honoré,"PET ser risiko for indrejsende, som truer land...",2023-08-04T08:27:31.820Z,0,"[Trusselsniveauet er uændret, men risikoen ved..."
373,Lokalt,Alexander Kloppenborg Lunde,24-årig kvinde følte sig befamlet hele tiden –...,2023-08-04T08:56:10.086Z,0,[Karen Kirstine Bøndergaard lider af paranoid ...
372,Udland,Ritzau,Andrew Tate er blevet løsladt,2023-08-04T09:33:43.244Z,0,"[Andrew Tate, der er tiltalt for menneskehande..."
371,Samfund,Jasmina Blichert,Politiet genindfører fast bemanding ved landeg...,2023-08-04T09:42:49.842Z,0,[Hvad sker der? Rigspolitiet har på opfordring...
370,Lokalt,Elisa Piccin,Mand død i oversvømmet tunnel,2023-08-04T11:28:11.891Z,0,[En 71-årig mand druknede torsdag i en oversvø...
369,Samfund,David Rue Honoré,Danske kunder føler sig snydt: 3760 kroner for...,2023-08-04T11:30:00.754Z,0,[En lang række danskere er opkrævet høje gebyr...


In [10]:
df.to_csv('Data.csv', index=False)
time.sleep(5)

In [11]:
df.to_csv('T:\Data.csv', index=False)
time.sleep(5)

In [17]:
import os
os.system("shutdown /s /t 10")

0

In [18]:
#os.system("shutdown /a")

0